In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 18s 1us/step


In [3]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [4]:
word_index = imdb.get_word_index()

# Os primeiros índices são reservados
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 2s 1us/step


In [5]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [6]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [7]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [69]:
# O formato de entrada é a contagem vocabulário usados pelas avaliações dos filmes (10000 palavras)
def get_model1(vocab_size, emb_dim, max_len=256) :
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, emb_dim))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.summary()
    
    return model

def get_model2(vocab_size, emb_dim, max_len=256) :
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size, emb_dim, input_length=max_len, mask_zero=True))
    model.add(keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(keras.layers.Dropout(0.05))
    model.add(keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(keras.layers.Conv1D(filters=16,  kernel_size=3, activation='relu', padding='same'))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dropout(0.05))

    # attention layer
    # model.add(attention())

    # output layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=128, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.summary()

    return model

In [10]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [27]:
partial_x_train[0], partial_y_train[0]

(array([   1,   13,  104,   14,    9,   31,    7,    4, 4343,    7,    4,
        3776, 3394,    2,  495,  103,  141,   87, 2048,   17,   76,    2,
          44,  164,  525,   13,  197,   14,   16,  338,    4,  177,   16,
        6118, 5253,    2,    2,    2,   21,   61, 1126,    2,   16,   15,
          36, 4621,   19,    4,    2,  157,    5,  605,   46,   49,    7,
           4,  297,    8,  276,   11,    4,  621,  837,  844,   10,   10,
          25,   43,   92,   81, 2282,    5,   95,  947,   19,    4,  297,
         806,   21,   15,    9,   43,  355,   13,  119,   49, 3636, 6951,
          43,   40,    4,  375,  415,   21,    2,   92,  947,   19,    4,
        2282, 1771,   14,    5,  106,    2, 1151,   48,   25,  181,    8,
          67,    6,  530, 9089, 1253,    7,    4,    2,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [28]:
decode_review(partial_x_train[0])

"<START> i think this is one of the weakest of the kenneth branagh <UNK> works after such great efforts as much <UNK> about nothing etc i thought this was poor the cast was weaker alicia <UNK> <UNK> <UNK> but my biggest <UNK> was that they messed with the <UNK> work and cut out some of the play to put in the musical dance sequences br br you just don't do shakespeare and then mess with the play sorry but that is just wrong i love some cole porter just like the next person but <UNK> don't mess with the shakespeare skip this and watch <UNK> books if you want to see a brilliant shakespearean adaptation of the <UNK> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <P

In [23]:
len(decode_review(partial_x_train[0]).split())

256

In [62]:
import pandas as pd
import json

def get_word_index(sentences) :
    all_tokens = []
    for txt in sentences :
        all_tokens += txt.split()

    tokens = pd.Series(all_tokens, range(len(all_tokens)), name="tokens")
    types = tokens.unique()
    word_index = {word : i for i, word in enumerate(["<pad>", "<unk>"] + list(types))}
    
    def decode_review(text):
        reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
        return ' '.join([reverse_word_index.get(i, '?') for i in text])
    
    return word_index, decode_review

def get_dataset(dataframe, word_index, max_len=256) :
    x_series, y_series = dataframe["text"], dataframe["class"]

    x_list = [txt.split() for txt in list(x_series)]
    
    x_seq = []
    for tknlst in x_list :
        seq = []
        for tkn in tknlst :
            try :
                seq.append(word_index[tkn])
            except KeyError :
                seq.append(word_index["<unk>"])
        seq = (seq + [0] * (max_len - len(seq))) if (len(seq) < max_len) else (seq[ : max_len])
        x_seq.append(seq)
    
    y_int = [1 if lb >= .5 else 0 for lb in list(y_series)]

    x = np.array(x_seq, dtype=int) 
    y = np.array(y_int, dtype=int) 

    return x, y

In [63]:
train_dataframe = pd.read_csv("../resources/datasets/StanfordSentimentTreebank/split/SST2Processed2-train.csv", index_col=0)
word_index, decode_review = get_word_index(train_dataframe["text"])
x_train, y_train = get_dataset(train_dataframe, word_index)
x_train.shape, y_train.shape

((8544, 256), (8544,))

In [64]:
x_train[1], y_train[1]

(array([ 2,  3, 32, 33, 34, 35,  3, 36, 35,  3, 37, 38,  5, 39, 40, 14, 18,
        41, 35, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 35, 53, 54, 55,
        56,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 

In [65]:
decode_review(x_train[1])

"<start> the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words can not adequately describe co writerdirector peter jackson's expanded vision of jrr tolkien's middle earth <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

In [66]:
val_dataframe = pd.read_csv("../resources/datasets/StanfordSentimentTreebank/split/SST2Processed2-dev.csv", index_col=0)
x_val, y_val = get_dataset(val_dataframe, word_index)
x_val.shape, y_val.shape

((1101, 256), (1101,))

In [73]:
model = get_model2(len(word_index), 100)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 256, 100)          1617700   
                                                                 
 conv1d_28 (Conv1D)          (None, 256, 128)          38528     
                                                                 
 conv1d_29 (Conv1D)          (None, 256, 64)           24640     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 128, 64)          0         
 1D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 128, 64)           0         
                                                                 
 conv1d_30 (Conv1D)          (None, 128, 32)           6176      
                                                     

In [71]:
history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
 3/17 [====>.........................] - ETA: 6s - loss: 0.6931 - accuracy: 0.4961

KeyboardInterrupt: 